# 1. Auto Labeling / Annotations 自动标注工具

In [ ]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(  
    data="path/to/new/data",
    det_model="yolov8n.pt",
    sam_model="mobile_sam.pt",
    device="cuda",
    output_dir="path/to/save_labels",
)

# 2. Convert COCO into YOLO Format

In [ ]:
from ultralytics.data.converter import convert_coco

convert_coco(  
    "../datasets/coco/annotations/",
    use_segments=False,
    use_keypoints=False,
    cls91to80=True,
)

# 3. Image Compression

In [ ]:
from pathlib import Path

from ultralytics.data.utils import compress_one_image

# 图像压缩: 在保持纵横比和质量的同时，将单个图像文件压缩到更小的尺寸。
for f in Path("path/to/dataset").rglob("*.jpg"):
    compress_one_image(f)

# 4. Auto-split Dataset

In [ ]:
#自动分割数据集: 将数据集自动分割成 训练/验证/测试 集，并将结果保存到 autosplit_*.txt 文件中
from ultralytics.data.utils import autosplit

autosplit(  
    path="path/to/images",
    weights=(0.9, 0.1, 0.0),  # (train, validation, test) fractional splits
    annotated_only=False,  # split only images with annotation file when True
)

# 5. Code Profiling

In [ ]:
from ultralytics.utils.ops import Profile

with Profile(device="cuda:0") as dt:
    pass  # operation to measure

print(dt)
# >>> "Elapsed time is 9.5367431640625e-07 s"

# 6. Ultralytics Supported Formats

In [ ]:
from ultralytics.data.utils import IMG_FORMATS, VID_FORMATS

print(IMG_FORMATS)
# {'tiff', 'pfm', 'bmp', 'mpo', 'dng', 'jpeg', 'png', 'webp', 'tif', 'jpg'}

print(VID_FORMATS)
# {'avi', 'mpg', 'wmv', 'mpeg', 'm4v', 'mov', 'mp4', 'asf', 'mkv', 'ts', 'gif', 'webm'}

# 7. Advanced Customization Low-Level

In [ ]:
from ultralytics.models.yolo.detect import DetectionTrainer

# Example override dictionary
overrides = {
    "batch_size": 16,
    "img_size": 640,
}

# DetectionTrainer
trainer = DetectionTrainer(overrides=overrides)
trainer.train()
trained_model = trainer.best  # get best model

In [ ]:
from ultralytics.models.yolo.detect import DetectionTrainer


class CustomTrainer(DetectionTrainer):
    def get_model(self, cfg, weights):
        """Loads a custom detection model given configuration and weight files."""
        ...


trainer = CustomTrainer(overrides={...})
trainer.train()

In [ ]:
from ultralytics.models.yolo.detect import DetectionTrainer
from ultralytics.nn.tasks import DetectionModel

import yaml

# Load your args.yaml content into a dictionary
with open('path/to/your/args.yaml', 'r') as file:
    yaml_content = yaml.safe_load(file)

class MyCustomModel(DetectionModel):
    def init_criterion(self):
        """Initializes the loss function and adds a callback for uploading the model to Google Drive every 10 epochs."""
        ...


class CustomTrainer(DetectionTrainer):
    def get_model(self, cfg, weights):
        """Returns a customized detection model instance configured with specified config and weights."""
        return MyCustomModel(...)


# callback to upload model weights
def log_model(trainer):
    """Logs the path of the last model weight used by the trainer."""
    last_weight_path = trainer.last
    print(last_weight_path)


trainer = CustomTrainer(overrides={...})
trainer.add_callback("on_train_epoch_end", log_model)  # Adds to existing callback
trainer.train()